# Webscraping S9
## Shotgun
## DIA1, A5

In [25]:
import re 
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import numpy as np
import time

## Projet:
Scrap the following: https://shotgun.live/fr/paris

On this page are all the links to the events themselves. I need to first get this link, and then access every link and scrap the event data.

## Part 1: Scrap links for all events

In [26]:
driver = webdriver.Chrome(executable_path='C:/chromedriver-win64/chromedriver.exe')

website_url = 'https://shotgun.live/fr/paris'
driver.get(website_url)

In [27]:
start_time = time.time()
duration = 120  

while time.time() - start_time < duration:
    try:
        bouton = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//div[contains(text(), "Voir plus")]'))
        )

        bouton.click()
        
    except Exception as e:
        print(f"Une exception s'est produite : {e}")

Une exception s'est produite : Message: element click intercepted: Element <div dir="auto" class="css-901oao r-jwli3a r-c321bz r-ubezar r-13uqrnb r-b88u0q r-1ikidpy r-rjixqe r-q4m81j r-tsynxw">...</div> is not clickable at point (518, 650). Other element would receive the click: <div id="CybotCookiebotDialog" name="CybotCookiebotDialog" role="dialog" aria-modal="true" aria-labelledby="CybotCookiebotDialogBodyContentTitle" tabindex="-1" lang="fr" dir="ltr" ng-non-bindable="" data-template="bottom" class="CybotEdge CybotCookiebotDialogActive" style="display: flex;">...</div>
  (Session info: chrome=120.0.6099.130)

Une exception s'est produite : Message: element click intercepted: Element <div dir="auto" class="css-901oao r-jwli3a r-c321bz r-ubezar r-13uqrnb r-b88u0q r-1ikidpy r-rjixqe r-q4m81j r-tsynxw">...</div> is not clickable at point (518, 650). Other element would receive the click: <div role="button" tabindex="0" class="css-18t94o4 css-1dbjc4n r-1awozwy r-1fpi9eo r-z2wwpe r-1yadl

In [28]:
page_html = driver.page_source
soup = BeautifulSoup(page_html, 'html.parser')

## Scrap

In [29]:
event_links = soup.find_all('a', class_='a-pressable-img')
df_shotgun = pd.DataFrame(columns=['title', 'date', 'hour', 'venue', 'price', 'link'])

for event_link in event_links:
    link = event_link.get('href')
    
    if not link.startswith('https://shotgun.live/events/'):
        continue
    
    row_data = {'link': link}
    
    title = event_link.find('div', class_='css-901oao')
    if title:
        title_text = title.get_text(strip=True)
        row_data['title'] = title_text

    location = event_link.find('div', class_='css-1hf3ou5')
    if location:
        location_text = location.get_text(strip=True)
        row_data['venue'] = location_text
        
    date_div = event_link.find('div', class_='r-d8nonl')
    time_div = event_link.find('div', class_='r-11zsu6o')
    price_div = event_link.find('div', class_='r-1n0xq6e')

    if date_div:
        date_text = date_div.get_text(strip=True)
        row_data['date'] = date_text
    
    separator_div = date_div.find_next('div', text=' | ')
    time_div = separator_div.find_next('div', class_='css-901oao')
    
    if time_div:    
        time_text = time_div.get_text(strip=True)
        row_data['hour'] = time_text

    if price_div:
        price_text = price_div.get_text(strip=True)
        row_data['price'] = price_text
        
    image_div = event_link.find('div', id=lambda x: x and x.startswith('event.'))
    if image_div:
        image_url = image_div.find('img').get('src')
        row_data['image_url'] = image_url

    df_shotgun = df_shotgun.append(row_data, ignore_index=True)

<ipython-input-29-855554fa6937>:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_shotgun = df_shotgun.append(row_data, ignore_index=True)
<ipython-input-29-855554fa6937>:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_shotgun = df_shotgun.append(row_data, ignore_index=True)
<ipython-input-29-855554fa6937>:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_shotgun = df_shotgun.append(row_data, ignore_index=True)
<ipython-input-29-855554fa6937>:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_shotgun = df_shotgun.append(row_data, ignore_index=True)
<ipython-input-29-855554fa6937>:46: FutureWarning: The frame.append method i

In [30]:
df_shotgun

,title,date,hour,venue,price,link,image_url
0,See The Light,Aujourd’hui,23:59,Magnum Klub,"8,99 €",https://shotgun.live/events/see-the-light,https://res.cloudinary.com/shotgun/image/uploa...
1,T7 Pass : Janvier/mai 2024,En cours,Jour 1/151,T7,NaN,https://shotgun.live/events/t-7-pass-janvier-m...,https://res.cloudinary.com/shotgun/image/uploa...
2,Sivilizasyon X Gambetta,Aujourd’hui,21:00,Sivilizasyon,Gratuit,https://shotgun.live/events/sivilizasyon-x-gam...,https://res.cloudinary.com/shotgun/image/uploa...
3,Rave Ablaze,Demain,23:59,Magnum Klub,"7,99 €",https://shotgun.live/events/rave-ablaze,https://res.cloudinary.com/shotgun/image/uploa...
4,Open Platines Du Liebe (02/01/24),Demain,22:00,Liebe,Gratuit,https://shotgun.live/events/open-platines-du-l...,https://res.cloudinary.com/shotgun/image/uploa...
...,...,...,...,...,...,...,...
574,La Dancing # Samedi 27 Juil,sam 27 juil.,23:55,Etoiles,"12,99 €",https://shotgun.live/events/la-dancing-samedi-...,https://res.cloudinary.com/shotgun/image/uploa...
575,La P'tite Fumée + Roots Zombie à Paris,mer 9 oct.,19:30,BASE PRODUCTIONS,"25,69 €",https://shotgun.live/events/la-p-tite-fumee-ro...,https://res.cloudinary.com/shotgun/image/uploa...
576,Rakoon @ élysée Montmartre,jeu 7 nov.,18:30,Bi:pole,"24,68 €",https://shotgun.live/events/rakoon-elysee-mont...,https://res.cloudinary.com/shotgun/image/uploa...
577,Afterparty With F1lthy Paris - Postponed,mar 31 déc.,23:00,MovidaClubParis,NaN,https://shotgun.live/events/afterparty-with-f-...,https://res.cloudinary.com/shotgun/image/uploa...


In [31]:
df_shotgun.shape

(579, 7)

In [32]:
df_shotgun.isnull().sum()

title         0
date          0
hour          0
venue         0
price        39
link          0
image_url     0
dtype: int64

## Part 2: Scrap event data for every event whose link was previously collected and stored in df_shotgun dataframe
We get the date and time of the event, the artists, the address of the venue, and the link to the venue's page

### Test scraping d'une page d'event
A mettre ensuite dans une boucle pour faire tous les événements du df_shotgun

In [9]:
event_url = 'https://shotgun.live/fr/events/coliseum-the-burning-lands'
driver.get(event_url)

In [10]:
page_source = driver.page_source
soup = BeautifulSoup(page_source,'html.parser')

In [11]:
event_title = soup.find('h1', class_='css-4rbku5 css-901oao r-jwli3a r-1o3oout r-adyw6z r-13uqrnb r-1vr29t4 r-19x0jo r-1wzrnnt r-tsynxw').text.strip()

venue_name_element = soup.find('h2', text='Lieu')
if venue_name_element:
    venue_name = venue_name_element.find_next('span').text.strip()
    venue_link = venue_name_element.find_next('a')['href']
    location = venue_name_element.find_next('span').find_next('span').text.strip()
else:
    venue_name = "Lieu secret"
    venue_link = "Lieu secret"
    location = "Lieu secret"
    
if location == "à":
    venue_name = "Lieu secret"
    venue_link = "Lieu secret"
    location = "Lieu secret"
    
date_and_time = soup.find('div', class_='css-901oao r-d8nonl r-c321bz r-ubezar r-13uqrnb r-majxgm r-rjixqe').text.strip()
date_and_time = date_and_time.replace(' De',',')
date_and_time = date_and_time.replace(' à ','-')
date, hour = map(str.strip, date_and_time.split(','))

organizer = soup.find('span', class_='css-901oao css-16my406 r-jwli3a r-c321bz r-ubezar r-13uqrnb r-b88u0q r-rjixqe').text.strip()

artist_name_elements = soup.find_all('div', class_='css-901oao css-cens5h r-1ub5np2 r-c321bz r-1b43r93 r-13uqrnb r-majxgm r-14yzgew r-kc8jnq r-nk90ef r-14q5wjw')
lineup = [element.text.strip() for element in artist_name_elements]
artists = ', '.join(lineup)

genres_elements = soup.find_all('div', class_='css-901oao r-1a7l8x0 r-c321bz r-1b43r93 r-13uqrnb r-majxgm r-1ikidpy r-rjixqe r-tsynxw')
genres_list = [element.text.strip() for element in genres_elements]
genres = ', '.join(genres_list)

event_info = (event_title, genres, date, hour, organizer, artists, location, venue_name, venue_link)

print(event_info)

('COLISEUM - THE BURNING LANDS', 'hardcore, hardtek, acidcore, hardstyle, hard techno, techno', 'sam 23 déc.', '23:00-07:00', 'Amnexia', 'Darktek, Teksa, A5KM, La Kajofol, Sasio, Rawcket, Flodzi', '100 Av. du Général Leclerc, 93500 Pantin, France', 'Nexus', 'https://www.google.com/maps/search/Nexus%2C%20100%20Av.%20du%20G%C3%A9n%C3%A9ral%20Leclerc%2C%2093500%20Pantin%2C%20France/@48.8971575,2.405313,17z')


In [12]:
premiere_ligne = df_shotgun.head(1).values.tolist()
ligne_en_tuple = tuple(premiere_ligne[0])
print(ligne_en_tuple)

('Coliseum - The Burning Lands', 'Aujourd’hui', '23:00', 'Amnexia', '20,99\xa0€', 'https://shotgun.live/events/coliseum-the-burning-lands')


In [13]:
df_shotgun

,title,date,hour,venue,price,link
0,Coliseum - The Burning Lands,Aujourd’hui,23:00,Amnexia,"20,99 €",https://shotgun.live/events/coliseum-the-burni...
1,"Ivory (afterlife/innervisions), Mimi X Fy, Sin...",Aujourd’hui,23:45,DUST,"36,75 €",https://shotgun.live/events/ivory-afterlife-in...
2,Pantheon Presents : Amine K & Davide Squillace...,Aujourd’hui,23:45,Maori Production,"21,00 €",https://shotgun.live/events/pantheon-presents-...
3,Un Noël Latino !,Aujourd’hui,23:00,LA BELLEVILLOISE,"12,10 €",https://shotgun.live/events/un-noel-latino
4,"Belle Epoque! W/ David Shaw And The Beat , Edo...",Aujourd’hui,23:00,Silencio Club,"15,00 €",https://shotgun.live/events/belle-epoque-w-dav...
...,...,...,...,...,...,...
691,La Dancing # Samedi 27 Juil,sam 27 juil. 2024,23:55,Etoiles,"12,99 €",https://shotgun.live/events/la-dancing-samedi-...
692,La P'tite Fumée + Roots Zombie à Paris,mer 9 oct. 2024,19:30,BASE PRODUCTIONS,"25,69 €",https://shotgun.live/events/la-p-tite-fumee-ro...
693,Rakoon @ élysée Montmartre,jeu 7 nov. 2024,18:30,Bi:pole,"24,68 €",https://shotgun.live/events/rakoon-elysee-mont...
694,Afterparty With F1lthy Paris - Postponed,mar 31 déc. 2024,23:00,MovidaClubParis,NaN,https://shotgun.live/events/afterparty-with-f-...


## Pour tous les événements

In [33]:
all_event_info = []

for event_url in df_shotgun['link']:
    
    driver.get(event_url)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source,'html.parser')

    event_title = soup.find('h1', class_='css-4rbku5 css-901oao r-jwli3a r-1o3oout r-adyw6z r-13uqrnb r-1vr29t4 r-19x0jo r-1wzrnnt r-tsynxw').text.strip()

    venue_name_element = soup.find('h2', text='Lieu')
    if venue_name_element:
        venue_name = venue_name_element.find_next('span').text.strip()
        venue_link = venue_name_element.find_next('a')['href']
        location = venue_name_element.find_next('span').find_next('span').text.strip()
    else:
        venue_name = "Lieu secret"
        venue_link = "Lieu secret"
        location = "Lieu secret"

    if location == "à":
        venue_name = "Lieu secret"
        venue_link = "Lieu secret"
        location = "Lieu secret"

    date_and_time = soup.find('div', class_='css-901oao r-d8nonl r-c321bz r-ubezar r-13uqrnb r-majxgm r-rjixqe').text.strip()

    organizer = soup.find('span', class_='css-901oao css-16my406 r-jwli3a r-c321bz r-ubezar r-13uqrnb r-b88u0q r-rjixqe').text.strip()

    artist_name_elements = soup.find_all('div', class_='css-901oao css-cens5h r-1ub5np2 r-c321bz r-1b43r93 r-13uqrnb r-majxgm r-14yzgew r-kc8jnq r-nk90ef r-14q5wjw')
    lineup = [element.text.strip() for element in artist_name_elements]
    artists = ', '.join(lineup)
    
    genres_elements = soup.find_all('div', class_='css-901oao r-1a7l8x0 r-c321bz r-1b43r93 r-13uqrnb r-majxgm r-1ikidpy r-rjixqe r-tsynxw')
    genres_list = [element.text.strip() for element in genres_elements]
    genres = ', '.join(genres_list)

    event_info = {
        'Event Title': event_title,
        'Genres': genres,
        'Date & Time': date_and_time,
        'Organizer': organizer,
        'Artists': artists,
        'Location': location,
        'Venue Name': venue_name,
        'Venue Link': venue_link
    }

    all_event_info.append(event_info)

shotgun_events_paris = pd.DataFrame(all_event_info)

AttributeError: 'NoneType' object has no attribute 'text'

In [35]:
shotgun_events_paris

NameError: name 'shotgun_events_paris' is not defined

In [36]:
df_shotgun = pd.read_csv('shotgun_events_paris.csv')
df_shotgun

,Unnamed: 0.1,Unnamed: 0,Event Title,Organizer,Artists,Location,Venue Name,Venue Link,Heure,Date,Price,Event Link,Genres
0,0,0,COLISEUM - THE BURNING LANDS,Amnexia,"Darktek, Teksa, A5KM, La Kajofol, Sasio, Rawck...","100 Av. du Général Leclerc, 93500 Pantin, France",Nexus,https://www.google.com/maps/search/Nexus%2C%20...,23:00 à 07:00,2023-12-23,"16,99 €",https://shotgun.live/events/coliseum-the-burni...,"hardcore, hardtek, acidcore, hardstyle, hard t..."
1,1,1,PANTHEON presents : AMINE K & DAVIDE SQUILLACE...,Maori Production,"Amine K (Moroko Loko), davide squillace, NAMU","8 Port des Invalides, 75007 Paris, France",Non mentionné,https://www.google.com/maps/search/8%20Port%20...,23:45 à 06:00,2023-12-23,"21,00 €",https://shotgun.live/events/pantheon-presents-...,NaN
2,2,2,"IVORY (AFTERLIFE/INNERVISIONS), MIMI x FY, SIN...",DUST,"Ivory, MĪMĪ x FY, Singi","Palais De Tokyo, 13 Av. du Président Wilson, 7...",YOYO,https://www.google.com/maps/search/YOYO%2C%20P...,23:45 à 06:00,2023-12-23,"31,50 €",https://shotgun.live/events/ivory-afterlife-in...,"deep house, techno, melodic house & techno, el..."
3,3,3,Un Noël latino !,LA BELLEVILLOISE,NaN,"19-21 Rue Boyer, 75020 Paris, France",La Bellevilloise,https://www.google.com/maps/search/La%20Bellev...,23:00 à 05:30,2023-12-23,"12,10 €",https://shotgun.live/events/un-noel-latino,"salsa, reggaeton, baile funk"
4,4,4,RÊVE PARTY - Péniche Cinéma - Techno to Trance,StereOrganic Productions,"Abiotiq - Bom Shanka Music, Auri0n, Otezuka, k...","59 Bd Macdonald, 75019 Paris, France",La Péniche Cinéma - Le Baruda,https://www.google.com/maps/search/La%20P%C3%A...,23:00 à 07:00,2023-12-23,"10,99 €",https://shotgun.live/events/reve-party-peniche...,"techno, hard techno, psytrance, dark psytrance"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
679,679,679,La Dancing # Samedi 27 Juil,Etoiles,"David Stepanoff, Marc Ryan, Sam Karlson","61 Rue du Château d'Eau, 75010 Paris, France",Les Étoiles,https://www.google.com/maps/search/Les%20%C3%8...,23:55 à 06:00,2024-07-27,"12,99 €",https://shotgun.live/events/la-dancing-samedi-...,"electro, hip hop, r&b, pop, dance, house"
680,680,680,LA P'TITE FUMÉE + Roots Zombie à Paris,BASE PRODUCTIONS,"La P'tite Fumée, Roots Zombie","80 Blvd Marguerite de Rochechouart, 75018 Pari...",Le Trianon,https://www.google.com/maps/search/Le%20Triano...,19:30 à 23:00,2024-10-09,"25,69 €",https://shotgun.live/events/la-p-tite-fumee-ro...,"electro, electronica, psytrance, techno"
681,681,681,RAKOON @ ÉLYSÉE MONTMARTRE,Bi:pole,Rakoon,"72 Blvd Marguerite de Rochechouart, 75018 Pari...",L'Élysée Montmartre,https://www.google.com/maps/search/L'%C3%89lys...,18:30 à 22:30,2024-11-07,"24,68 €",https://shotgun.live/events/rakoon-elysee-mont...,"dub, house, electro, pop, synthpop"
682,682,682,AFTERPARTY with F1LTHY PARIS - POSTPONED,MovidaClubParis,NaN,"32 Av. Corentin Cariou, 75019 Paris, France",MOVIDA CLUB PARIS,https://www.google.com/maps/search/MOVIDA%20CL...,23:00 à 23:59,2024-12-31,NaN,https://shotgun.live/events/afterparty-with-f-...,NaN


In [25]:
all_genres_infos = []

for event_url in df_shotgun['Event Link']:
    
    driver.get(event_url)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source,'html.parser')
    
    genres_elements = soup.find_all('div', class_='css-901oao r-1a7l8x0 r-c321bz r-1b43r93 r-13uqrnb r-majxgm r-1ikidpy r-rjixqe r-tsynxw')
    genres_list = [element.text.strip() for element in genres_elements]
    genres = ', '.join(genres_list)

    genres_infos = {
        'Genres': genres
    }

    all_genres_infos.append(genres_infos)

genres_df = pd.DataFrame(all_genres_infos)

In [26]:
genres_df

,Genres
0,"hardcore, hardtek, acidcore, hardstyle, hard t..."
1,
2,"deep house, techno, melodic house & techno, el..."
3,"salsa, reggaeton, baile funk"
4,"techno, hard techno, psytrance, dark psytrance"
...,...
679,"electro, hip hop, r&b, pop, dance, house"
680,"electro, electronica, psytrance, techno"
681,"dub, house, electro, pop, synthpop"
682,


In [ ]:
all_event_info

In [ ]:
shotgun_events_paris

In [ ]:
shotgun_events_paris.isnull().sum()

In [ ]:
shotgun_events_paris.info()

In [ ]:
test = shotgun_events_paris.copy()
test.head()

In [ ]:
test = shotgun_events_paris.copy()
test.head()

month_abbreviations = {
    'Jan': 1, 'janv': 1,
    'Feb': 2, 'févr': 2,
    'Mar': 3, 'mars': 3,
    'Apr': 4, 'avr': 4,
    'May': 5, 'mai': 5,
    'Jun': 6, 'juin': 6,
    'Jul': 7, 'juil': 7,
    'Aug': 8, 'août': 8,
    'Sep': 9, 'sept': 9,
    'Oct': 10, 'oct': 10,
    'Nov': 11, 'nov': 11,
    'Dec': 12, 'déc': 12
}

def extract_month(date):
    if pd.notnull(date):
        for month, number in month_abbreviations.items():
            if month in date:
                return number
    return None

test['Month'] = test['Date & Time'].apply(lambda x: extract_month(x))

days_list = [' 1 ', ' 2 ', ' 3 ', ' 4 ', ' 5 ', ' 6 ', ' 7 ', ' 8 ', ' 9 ', ' 10 ',
             ' 11 ', ' 12 ', ' 13 ', ' 14 ', ' 15 ', ' 16 ', ' 17 ', ' 18 ', ' 19 ',
             ' 20 ', ' 21 ', ' 22 ', ' 23 ', ' 24 ', ' 25 ', ' 26 ', ' 27 ', ' 28 ',
             ' 29 ', ' 30 ', ' 31 ']

def extract_day(date):
    if pd.notnull(date):
        for day in days_list:
            if day in date:
                return int(day.strip())
    return None

test['Day'] = test['Date & Time'].apply(lambda x: extract_day(x))

def extract_hour(date_time):
    if pd.notnull(date_time):
        match = re.search(r'(?:From|De|à)\s*(.*$)', date_time)
        if match:
            return match.group(1).strip()
    return None

test['Heure'] = test['Date & Time'].apply(lambda x: extract_hour(x))
test['Année'] = test['Date & Time'].apply(lambda x: 2024 if pd.notnull(x) and '2024' in x else 2023)

test['Day'] = pd.to_numeric(test['Day'], errors='coerce', downcast='integer')
test['Année'] = pd.to_numeric(test['Année'], errors='coerce', downcast='integer')
test['Month'] = pd.to_numeric(test['Month'], errors='coerce', downcast='integer')

test = test.rename(columns={'Année': 'year', 'Month': 'month', 'Day': 'day'})
test['Date'] = pd.to_datetime(test[['year', 'month', 'day']], errors='coerce')

test = test.drop(['Date & Time','month', 'day', 'year'], axis=1)

prices_links = df_shotgun[['price','link']]
df_final = pd.concat([test, prices_links], axis=1)
df_final = df_final.rename(columns={'price': 'Price', 'link': 'Event Link'})

df_final.loc[df_final['Location'] == 'Pourquoi Shotgun\xa0?', 'Location'] = df_final['Venue Name']
df_final['Venue Name'] = np.where(df_final['Location'] == df_final['Venue Name'], 'Non mentionné', df_final['Venue Name'])

df_final

In [34]:
df_shotgun
df_final = pd.concat([df_shotgun, genres_df], axis=1)
df_final

,Unnamed: 0,Event Title,Organizer,Artists,Location,Venue Name,Venue Link,Heure,Date,Price,Event Link,Genres
0,0,COLISEUM - THE BURNING LANDS,Amnexia,"Darktek, Teksa, A5KM, La Kajofol, Sasio, Rawck...","100 Av. du Général Leclerc, 93500 Pantin, France",Nexus,https://www.google.com/maps/search/Nexus%2C%20...,23:00 à 07:00,2023-12-23,"16,99 €",https://shotgun.live/events/coliseum-the-burni...,"hardcore, hardtek, acidcore, hardstyle, hard t..."
1,1,PANTHEON presents : AMINE K & DAVIDE SQUILLACE...,Maori Production,"Amine K (Moroko Loko), davide squillace, NAMU","8 Port des Invalides, 75007 Paris, France",Non mentionné,https://www.google.com/maps/search/8%20Port%20...,23:45 à 06:00,2023-12-23,"21,00 €",https://shotgun.live/events/pantheon-presents-...,
2,2,"IVORY (AFTERLIFE/INNERVISIONS), MIMI x FY, SIN...",DUST,"Ivory, MĪMĪ x FY, Singi","Palais De Tokyo, 13 Av. du Président Wilson, 7...",YOYO,https://www.google.com/maps/search/YOYO%2C%20P...,23:45 à 06:00,2023-12-23,"31,50 €",https://shotgun.live/events/ivory-afterlife-in...,"deep house, techno, melodic house & techno, el..."
3,3,Un Noël latino !,LA BELLEVILLOISE,NaN,"19-21 Rue Boyer, 75020 Paris, France",La Bellevilloise,https://www.google.com/maps/search/La%20Bellev...,23:00 à 05:30,2023-12-23,"12,10 €",https://shotgun.live/events/un-noel-latino,"salsa, reggaeton, baile funk"
4,4,RÊVE PARTY - Péniche Cinéma - Techno to Trance,StereOrganic Productions,"Abiotiq - Bom Shanka Music, Auri0n, Otezuka, k...","59 Bd Macdonald, 75019 Paris, France",La Péniche Cinéma - Le Baruda,https://www.google.com/maps/search/La%20P%C3%A...,23:00 à 07:00,2023-12-23,"10,99 €",https://shotgun.live/events/reve-party-peniche...,"techno, hard techno, psytrance, dark psytrance"
...,...,...,...,...,...,...,...,...,...,...,...,...
679,679,La Dancing # Samedi 27 Juil,Etoiles,"David Stepanoff, Marc Ryan, Sam Karlson","61 Rue du Château d'Eau, 75010 Paris, France",Les Étoiles,https://www.google.com/maps/search/Les%20%C3%8...,23:55 à 06:00,2024-07-27,"12,99 €",https://shotgun.live/events/la-dancing-samedi-...,"electro, hip hop, r&b, pop, dance, house"
680,680,LA P'TITE FUMÉE + Roots Zombie à Paris,BASE PRODUCTIONS,"La P'tite Fumée, Roots Zombie","80 Blvd Marguerite de Rochechouart, 75018 Pari...",Le Trianon,https://www.google.com/maps/search/Le%20Triano...,19:30 à 23:00,2024-10-09,"25,69 €",https://shotgun.live/events/la-p-tite-fumee-ro...,"electro, electronica, psytrance, techno"
681,681,RAKOON @ ÉLYSÉE MONTMARTRE,Bi:pole,Rakoon,"72 Blvd Marguerite de Rochechouart, 75018 Pari...",L'Élysée Montmartre,https://www.google.com/maps/search/L'%C3%89lys...,18:30 à 22:30,2024-11-07,"24,68 €",https://shotgun.live/events/rakoon-elysee-mont...,"dub, house, electro, pop, synthpop"
682,682,AFTERPARTY with F1LTHY PARIS - POSTPONED,MovidaClubParis,NaN,"32 Av. Corentin Cariou, 75019 Paris, France",MOVIDA CLUB PARIS,https://www.google.com/maps/search/MOVIDA%20CL...,23:00 à 23:59,2024-12-31,NaN,https://shotgun.live/events/afterparty-with-f-...,


In [35]:
df_final.to_csv('shotgun_events_paris.csv')

In [37]:
## add image


In [39]:
shotgun = pd.read_csv('shotgun_events_paris.csv')
shotgun

,Unnamed: 0.1,Unnamed: 0,Event Title,Organizer,Artists,Location,Venue Name,Venue Link,Heure,Date,Price,Event Link,Genres
0,0,0,COLISEUM - THE BURNING LANDS,Amnexia,"Darktek, Teksa, A5KM, La Kajofol, Sasio, Rawck...","100 Av. du Général Leclerc, 93500 Pantin, France",Nexus,https://www.google.com/maps/search/Nexus%2C%20...,23:00 à 07:00,2023-12-23,"16,99 €",https://shotgun.live/events/coliseum-the-burni...,"hardcore, hardtek, acidcore, hardstyle, hard t..."
1,1,1,PANTHEON presents : AMINE K & DAVIDE SQUILLACE...,Maori Production,"Amine K (Moroko Loko), davide squillace, NAMU","8 Port des Invalides, 75007 Paris, France",Non mentionné,https://www.google.com/maps/search/8%20Port%20...,23:45 à 06:00,2023-12-23,"21,00 €",https://shotgun.live/events/pantheon-presents-...,NaN
2,2,2,"IVORY (AFTERLIFE/INNERVISIONS), MIMI x FY, SIN...",DUST,"Ivory, MĪMĪ x FY, Singi","Palais De Tokyo, 13 Av. du Président Wilson, 7...",YOYO,https://www.google.com/maps/search/YOYO%2C%20P...,23:45 à 06:00,2023-12-23,"31,50 €",https://shotgun.live/events/ivory-afterlife-in...,"deep house, techno, melodic house & techno, el..."
3,3,3,Un Noël latino !,LA BELLEVILLOISE,NaN,"19-21 Rue Boyer, 75020 Paris, France",La Bellevilloise,https://www.google.com/maps/search/La%20Bellev...,23:00 à 05:30,2023-12-23,"12,10 €",https://shotgun.live/events/un-noel-latino,"salsa, reggaeton, baile funk"
4,4,4,RÊVE PARTY - Péniche Cinéma - Techno to Trance,StereOrganic Productions,"Abiotiq - Bom Shanka Music, Auri0n, Otezuka, k...","59 Bd Macdonald, 75019 Paris, France",La Péniche Cinéma - Le Baruda,https://www.google.com/maps/search/La%20P%C3%A...,23:00 à 07:00,2023-12-23,"10,99 €",https://shotgun.live/events/reve-party-peniche...,"techno, hard techno, psytrance, dark psytrance"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
679,679,679,La Dancing # Samedi 27 Juil,Etoiles,"David Stepanoff, Marc Ryan, Sam Karlson","61 Rue du Château d'Eau, 75010 Paris, France",Les Étoiles,https://www.google.com/maps/search/Les%20%C3%8...,23:55 à 06:00,2024-07-27,"12,99 €",https://shotgun.live/events/la-dancing-samedi-...,"electro, hip hop, r&b, pop, dance, house"
680,680,680,LA P'TITE FUMÉE + Roots Zombie à Paris,BASE PRODUCTIONS,"La P'tite Fumée, Roots Zombie","80 Blvd Marguerite de Rochechouart, 75018 Pari...",Le Trianon,https://www.google.com/maps/search/Le%20Triano...,19:30 à 23:00,2024-10-09,"25,69 €",https://shotgun.live/events/la-p-tite-fumee-ro...,"electro, electronica, psytrance, techno"
681,681,681,RAKOON @ ÉLYSÉE MONTMARTRE,Bi:pole,Rakoon,"72 Blvd Marguerite de Rochechouart, 75018 Pari...",L'Élysée Montmartre,https://www.google.com/maps/search/L'%C3%89lys...,18:30 à 22:30,2024-11-07,"24,68 €",https://shotgun.live/events/rakoon-elysee-mont...,"dub, house, electro, pop, synthpop"
682,682,682,AFTERPARTY with F1LTHY PARIS - POSTPONED,MovidaClubParis,NaN,"32 Av. Corentin Cariou, 75019 Paris, France",MOVIDA CLUB PARIS,https://www.google.com/maps/search/MOVIDA%20CL...,23:00 à 23:59,2024-12-31,NaN,https://shotgun.live/events/afterparty-with-f-...,NaN


In [50]:
from urllib.parse import urljoin

images = []

top3 = shotgun.head(3)

for event_url in top3['Event Link']:
    driver.get(event_url)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    image_element = soup.select_one('.css-1dbjc4n.r-1awozwy.r-1777fci.r-zso239.r-1l7z4oj.r-95jzfe img')

    if image_element:
        image_link = image_element.get('src')
        image = {
            'Image': image_link
        }
        images.append(image)
    else:
        print(f"Aucun élément img trouvé pour l'URL : {event_url}")

df_images = pd.DataFrame(images)
df_images

Aucun élément img trouvé pour l'URL : https://shotgun.live/events/coliseum-the-burning-lands
Aucun élément img trouvé pour l'URL : https://shotgun.live/events/pantheon-presents-amine-k-davide-squillace-23-12
Aucun élément img trouvé pour l'URL : https://shotgun.live/events/ivory-afterlife-innervisions-mimi-x-fy-singi-i-dust


""
